# LSTMs for Blink Recognition Time Series Classification

This notebook follows the example as shown in the following website:<br>
https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/


In [135]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import Polynomial
from sklearn.linear_model import LinearRegression
from pandas import read_csv
from numpy import dstack, mean, std
from tensorflow import keras
from tensorflow.keras import losses
from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D
from tensorflow.keras.utils import to_categorical

# Parameters

In [136]:
timesteps = 1000
shift_amount = 10
poly_regression_degree = 4

# Basic LSTM Model

The section is divided into the following sections:
1. Load the Data
2. Fit and Evaluate a Model
3. Summarize Results
4. Final Examination

## Load the Data

In [137]:
# load a channel, skip the first 6 rows, replace NaN with rolling average
def load_channel(filepath, columnindex):
  dataframe = read_csv(filepath, usecols=[columnindex], names=['voltage'], header=None, delim_whitespace=True)

  mask = dataframe['voltage'].isna()  # Create Boolean mask for Nan values

  x = np.arange(len(dataframe['voltage']))
  y = dataframe['voltage'].values
  p = Polynomial.fit(x[~mask], y[~mask], deg=poly_regression_degree)  # Fit polynomial regression model

  dataframe.loc[mask, 'voltage'] = p(x[mask])  # Replace NaN values with polynomial regression model values

  # Convert the one column DataFrame to a numpy array
  data = dataframe.to_numpy().squeeze()

  # Calculate the number of rows in the new DataFrame
  n_rows = (len(data) - timesteps) // shift_amount + 1

  # Initilize the new DataFrame
  new_df = pd.DataFrame(np.zeros((n_rows, timesteps)))

  # Fill the new DataFrame with shifted windows of data
  for i in range(n_rows):
      start = i * shift_amount
      end = start + timesteps
      new_df.iloc[i,:] = data[start:end]
  
  # print(dataframe)
  # print(new_df)
  return new_df.values

In [138]:
# load all channels into a 3D array of [samples, timesteps, features]
# samples = number of rows
# timesteps = size of rolling window
# features = number of channels
def load_group(filename):
  loaded = list()
  num_channels = 1
  with open(filename, 'r') as f:
    first_line = f.readline()
    num_channels = len(first_line.split())

  for channel in range(num_channels):
    data = load_channel(filename, channel)
    loaded.append(data)
  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  return loaded, num_channels

In [139]:
def replace_comments(val):
	if val.startswith('#'):
			return 2
	else:
			return 0

# load a dataset group, such as train or test
def load_dataset_group(filename):
	# load input data
	X, num_channels = load_group(filename)

	# load class output
	# 1 = punch
	# 2 = retract
	file = open(filename, 'r')
	raw_y = pd.DataFrame(np.zeros(len(file.readlines())), columns=['label'])
	file.seek(0)
	armExtended = False
	index = 0
	for line in file:
		line = line.strip()
		if '#' in line:
				if armExtended:
						raw_y.loc[index, 'label'] = 2
				else:
						raw_y.loc[index, 'label'] = 1
				armExtended = not armExtended
		index += 1
	file.close()
	# Convert the one column DataFrame to a numpy array
	data = raw_y.to_numpy().squeeze()

	 # Calculate the number of rows in the new DataFrame
	n_rows = (len(data) - timesteps) // shift_amount + 1

  # Initilize the new DataFrame
	y = pd.DataFrame(np.zeros(n_rows), columns=['label'])
	print(y.shape)

	detection_range = (int(timesteps * 0.1), int(timesteps * 0.4))
	start_detection = detection_range[0]
	end_detection = detection_range[1]
  # Fill the new DataFrame with shifted windows of data
	for i in range(n_rows):
		start = i * shift_amount
		end = start + timesteps
		window = data[start:end]
		if np.any(window[start_detection:end_detection] != 0):
			y.loc[i, 'label'] = window[start_detection:end_detection][np.nonzero(window[start_detection:end_detection])][0]
		else:
			y.loc[i, 'label'] = 0

	return X, y

In [140]:
# load the dataset, returns train and test X and y elements
def load_dataset(filename):	
	# load data
	X, y = load_dataset_group(filename)
	train_size = int(len(X) * 0.7)
  
	# split into train and test
	trainX, testX = X[0:train_size], X[train_size:]
	trainy, testy = y[0:train_size], y[train_size:]
 
	print(trainX.shape, trainy.shape)
	print(testX.shape, testy.shape)
 
	# # zero-offset class values
	# trainy = trainy - 1
	# testy = testy - 1
 
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

## Fit and Evaluate a Model

In [141]:
# # fit and evaluate a model
# def evaluate_model(trainX, trainy, testX, testy):
#   verbose, epochs, batch_size = 0, 15, 64
#   n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
#   model = keras.Sequential([
#     LSTM(100, input_shape=(n_timesteps,n_features), return_sequences=True),
#     Dropout(0.2),
#     LSTM(50),
#     Dropout(0.2),
#     Dense(100, activation='relu'),
#     Dropout(0.2),
#     Dense(n_outputs, activation='softmax')
#   ])
#   model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#   # fit network
#   early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#   history = model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_split=0.2, callbacks=[early_stop])
#   # plot learning curves
#   plt.plot(history.history['accuracy'], label='train')
#   plt.plot(history.history['val_accuracy'], label='test')
#   plt.legend()
#   plt.show()
#   # evaluate model
#   _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
#   return accuracy


## Summarize Results

In [142]:
# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [143]:
# run an experiment
def run_experiment(repeats=5):
  # load data
  trainX, trainy, testX, testy = load_dataset("Data/ThrowingPunch-Frontal-Elbow-Outward-ClosedElbow.txt")
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

## Final Examination

In [144]:
# run_experiment()

# CNN-LSTM Network Model

In [145]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  verbose, epochs, batch_size = 0, 50, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  # reshape data into time steps of sub-sequences
  n_steps, n_length = 10, 100
  trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
  testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
  # define model
  model = keras.Sequential([
    TimeDistributed(Conv1D(filters=64, kernel_size=3,
              activation='relu'), input_shape=(None, n_length, n_features)),
    TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')),
    TimeDistributed(Dropout(0.5)),
    TimeDistributed(MaxPooling1D(pool_size=2)),
    TimeDistributed(Flatten()),
    LSTM(100, return_sequences=True),
    Dropout(0.5),
    LSTM(100),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dense(n_outputs, activation='softmax')
  ])
  model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])
  # fit network
  early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
  history = model.fit(trainX, trainy, epochs=epochs,
            batch_size=batch_size, verbose=verbose, validation_split=0.2, callbacks=[early_stopping])
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy


In [146]:
run_experiment()

(115496, 1)
(80847, 1000, 2) (80847, 1)
(34649, 1000, 2) (34649, 1)
(80847, 1000, 2) (80847, 3) (34649, 1000, 2) (34649, 3)
Restoring model weights from the end of the best epoch: 4.
Epoch 7: early stopping
>#1: 78.199
Restoring model weights from the end of the best epoch: 2.
Epoch 5: early stopping
>#2: 78.799
Restoring model weights from the end of the best epoch: 12.
Epoch 15: early stopping
>#3: 78.894
Restoring model weights from the end of the best epoch: 4.
Epoch 7: early stopping
>#4: 77.041
Restoring model weights from the end of the best epoch: 7.
Epoch 10: early stopping
>#5: 77.439
[78.19850444793701, 78.79881262779236, 78.89404892921448, 77.04118490219116, 77.43946313858032]
Accuracy: 78.074% (+/-0.732)


# ConvLSTM Network Model

A further extension of the CNN LSTM idea is to perform the convolutions of the CNN (e.g. how the CNN reads the input sequence data) as part of the LSTM.

In [147]:
# # fit and evaluate a model
# def evaluate_model(trainX, trainy, testX, testy):
#   # define model
#   verbose, epochs, batch_size = 'auto', 25, 64
#   n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
#   # reshape into subsequences (samples, time steps, rows, cols, channels)
#   n_steps, n_length = 10, 100
#   trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
#   testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
#   # define model
#   model = keras.Sequential([
#     ConvLSTM2D(filters=64, kernel_size=(3, 3),
#               activation='relu', input_shape=(n_steps, 1, n_length, n_features), padding='same', return_sequences=True),
#     ConvLSTM2D(filters=64, kernel_size=(3, 3),
#               activation='relu', padding='same', return_sequences=True),
#     ConvLSTM2D(filters=64, kernel_size=(3, 3),
#               activation='relu', padding='same', return_sequences=False),
#     Dropout(0.5),
#     Flatten(),
#     Dense(100, activation='relu'),
#     Dense(n_outputs, activation='softmax')
#   ])
#   model.compile(loss='categorical_crossentropy',
#                 optimizer='adam', metrics=['accuracy'])
#   # fit network
#   model.fit(trainX, trainy, epochs=epochs,
#             batch_size=batch_size, verbose=verbose, use_multiprocessing=True)
#   # evaluate model
#   _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
#   return accuracy

In [148]:
# run_experiment()

In [149]:
# License:
# ========
# Use of this dataset in publications must be acknowledged by referencing the following publication [1] 

# [1] Davide Anguita, Alessandro Ghio, Luca Oneto, Xavier Parra and Jorge L. Reyes-Ortiz.
# A Public Domain Dataset for Human Activity Recognition Using Smartphones.
# 21th European Symposium on Artificial Neural Networks, Computational Intelligence
# and Machine Learning, ESANN 2013. Bruges, Belgium 24-26 April 2013. 

# This dataset is distributed AS-IS and no responsibility implied or
# explicit can be addressed to the authors or their institutions for
# its use or misuse. Any commercial use is prohibited.

# Other Related Publications:
# ===========================
# [2] Davide Anguita, Alessandro Ghio, Luca Oneto, Xavier Parra, Jorge L.
# Reyes-Ortiz.  Energy Efficient Smartphone-Based Activity Recognition
# using Fixed-Point Arithmetic. Journal of Universal Computer Science.
# Special Issue in Ambient Assisted Living: Home Care.   Volume 19, Issue 9. May 2013

# [3] Davide Anguita, Alessandro Ghio, Luca Oneto, Xavier Parra and
# Jorge L. Reyes-Ortiz. Human Activity Recognition on Smartphones
# using a Multiclass Hardware-Friendly Support Vector Machine. 4th
# International Workshop of Ambient Assited Living, IWAAL 2012,
# Vitoria-Gasteiz, Spain, December 3-5, 2012. Proceedings. Lecture
# Notes in Computer Science 2012, pp 216-223. 

# [4] Jorge Luis Reyes-Ortiz, Alessandro Ghio, Xavier Parra-Llanas,
# Davide Anguita, Joan Cabestany, Andreu Català. Human Activity and
# Motion Disorder Recognition: Towards Smarter Interactive Cognitive
# Environments. 21th European Symposium on Artificial Neural Networks,
# Computational Intelligence and Machine Learning, ESANN 2013. Bruges,
# Belgium 24-26 April 2013.  

# ==================================================================================================
# Jorge L. Reyes-Ortiz, Alessandro Ghio, Luca Oneto, Davide Anguita and Xavier Parra. November 2013.